In [122]:
# states:
UNINFECTED = 0
INFECTED = 1
RECOVERED = 3
DEAD = 4

# parameters:
RECOVERY_TIME = 21
INFECTION_CHANCE = .5
DEATH_CHANCE = .02
GRID_SIZE = 20
INITIAL_INFECTED = 1

In [123]:
from attr import attrs, attrib
import random

@attrs
class Person:
    status = attrib(default=UNINFECTED)
    infection_days = attrib(default=0)
    _becoming_infected = attrib(default=False)
    
    def become_infected(self):
        if self.status is UNINFECTED:
            self._becoming_infected=True
            
    def infect_neighbors(self, neighbors):
        if self.status is INFECTED:
            for neighbor in neighbors:
                if random.random() < INFECTION_CHANCE:
                    neighbor.become_infected()
                    
    def progression_step(self):
        if self._becoming_infected:
            self.status = INFECTED
            infection_days = 0
            self._becoming_infected = False
        
        if self.status is INFECTED:
            self.infection_days += 1
            
        if self.infection_days >= RECOVERY_TIME:
            if random.random() < DEATH_CHANCE:
                self.status = DEAD
            else:
                self.status = RECOVERED
                
    def __str__(self):
        return {
            UNINFECTED: '😀',
            INFECTED: '🤢',
            RECOVERED: '😎',
            DEAD: '💀'
            
        }[self.status]
                
class Grid:
    def __init__(self, size):
        self.rows = [[Person(status=UNINFECTED) 
                      for _ in range(GRID_SIZE)] 
                         for _ in range(GRID_SIZE)]
    
    def print(self):
        for row in self.rows:
            print("".join([str(person) for person in row]))
            
    def stats(self):
        stats = {
            UNINFECTED: 0,
            INFECTED: 0,
            RECOVERED: 0,
            DEAD: 0,
        }
        for row in self.rows:
            for person in row:
                stats[person.status] += 1
        return stats
    
    def people(self):
        for column in range(GRID_SIZE):
            for row in range(GRID_SIZE):
                neighbor_indices = grid_neighbors(column, row)
                person = self.rows[row][column]
                neighbors = [self.rows[row][column] for column, row in neighbor_indices]
                yield column, row, person, neighbors
                
    def step(self, num_steps=1):
        for _ in range(num_steps):
            for column, row, person, neighbors in self.people():
                person.infect_neighbors(neighbors)

            for column, row, person, neighbors in self.people():
                person.progression_step()
                
def grid_neighbors(column, row):
    neighbors = []
    if row > 0:
        neighbors.append((column, row - 1))
    if column > 0:
        neighbors.append((column - 1, row))
    if row < GRID_SIZE - 1:
        neighbors.append((column, row + 1))
    if column < GRID_SIZE - 1:
        neighbors.append((column + 1, row))
    return neighbors
        
# print(grid_neighbors(0, 0))
# print(grid_neighbors(GRID_SIZE - 1, GRID_SIZE - 1))

In [124]:
# set up the grid
grid = Grid(GRID_SIZE)
_mid_cell = GRID_SIZE // 2
grid.rows[_mid_cell][_mid_cell].status = INFECTED


In [127]:
# do a step or steps manually for debugging etc.
# grid.step()
# grid.print()